# Segmenting and Clustering Neighborhoods in Toronto part 1

## Previous part of the assignment

In [3]:
import bs4
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source, 'lxml')
table=soup.find('table',{'class':'wikitable sortable'})

A=[]
B=[]
C=[]

for row in table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells)==3: #Only extract table body not heading
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        
df=pd.DataFrame(A,columns=['PostCode'])
df['Borough']=B
df['Neighbourhood']=C

df_2=df[~df.Borough.str.contains("Not assigned")]
df_2=df_2.reset_index(drop=True)
df_3=df_2.groupby(['PostCode', 'Borough'])['Neighbourhood'].apply(lambda text: ''.join(text.to_string(index=False))).str.replace('(\\n)', ', ').reset_index()
df_4= df_3.Neighbourhood.replace('Not assigned', df_3.Borough, regex=True)
df_5=df_3.drop(columns=['Neighbourhood'])
df_6=df_5.assign(Neighbourhood=df_4.values)
df_6.shape

(103, 3)

## Second part of the assignment

Charges the necessary modules for this part of the assignement

In [1]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    openssl-1.0.2p             |    h14c3975_1002         3.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    conda-4.6.0                |        py36_1000         878 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0         conda-forge
    geopy:         1.18.1-py_0       conda-forge

The following packages will be UPDATED:

    conda:         4.5.12-py36_1000  conda-forge --> 4.6.0-py36_1000      conda-forge
    o

Changes df_6 to post_code_toronto for better clarity

In [4]:
postcode_toronto=df_6

Splits 'postcode_toronto' to keep just the Neighbourhood part

In [6]:
split_neighbourhood=postcode_toronto["Neighbourhood"].str.split(",", n = 1, expand = True) 

Retrieves the coordinate based on the name of the neighbourhood

In [ ]:
def geol_lat_long(address):

    geolocator = Nominatim(user_agent="my-application")
    location = geolocator.geocode(address+',Toronto')
    if location:
        latitude = location.latitude
        longitude = location.longitude
        return latitude,longitude
    else:
        return('0,0')
    print('Done!')

Adds the latitudes and longitudes in a single column

In [8]:
split_neighbourhood['lat-long'] = split_neighbourhood[0].apply(geol_lat_long)

Selects the column 'lat-long' from the previous object

In [15]:
split_lat_long=split_neighbourhood['lat-long']

Transforms 'split_lat_long' to a string

In [17]:
split_lat_long= split_lat_long.astype('str')

Removes the parenthesis from the latitude and longitude column, splits the columns in two after the ',' and creates two columns 'Latitude' and 'Longitude'

In [18]:
split_lat_long=split_lat_long.str.strip('()') \
    .str.split(',', expand=True) \
    .rename(columns={0:'Latitude', 1:'Longitude'})

Adds the latitude and longitudes columns to the main dataframe created in the first part of the assignement

In [19]:
df_7=df_6.assign(Latitude=split_lat_long['Latitude'].values)
df_8=df_7.assign(Longitude=split_lat_long['Longitude'].values)

Displays the dataframe with the latitudes and the longitudes

In [21]:
df_8

,PostCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8049304,-79.1658374
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7901172,-79.1733344
2,M1E,Scarborough,"Guildwood, , Morningside, West Hill",43.7548985,-79.1977755
3,M1G,Scarborough,Woburn,43.7598243,-79.2252908
4,M1H,Scarborough,Cedarbrae,43.75646655,-79.226692442588
5,M1J,Scarborough,Scarborough Village,43.7437422,-79.2116324
6,M1K,Scarborough,"East Birchmount Park, , Ionview,...",43.7141672,-79.271109
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7088231,-79.2959856
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarbor...",43.7219387,-79.2362324
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.70211215,-79.2600905301568


For the next part due to problems to retrieve some of the coordinates, the csv file containing the coordinates (http://cocl.us/Geospatial_data) will be used